<a href="https://colab.research.google.com/github/llinzy/nlp_deep_learning/blob/master/blstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import itertools
import glob

In [17]:
%cd drive/My Drive/nlp_deeplearning

/content/drive/My Drive/nlp_deeplearning


In [18]:
glob_1=glob.glob('nlp_deep_learning/glovesplit4/*.csv')
glob_2=glob.glob('nlp_deep_learning/glovesplit4.2/*.csv')

In [19]:
glob_=list(itertools.chain.from_iterable([glob_1,glob_2]))
len(glob_)

131

In [20]:
from pandas.io.parsers import ParserError

In [21]:
errors=[]
for i in glob_:
  try:
    d.append(pd.read_csv(i))
  except ParserError as error:
    errors.append(str(error))

In [9]:
import re

In [10]:
errors_=[int(p[0]) for p in [re.findall(r'\d{1,6}', str(i)) for i in errors]]

In [11]:
d=[]
for i in glob_:
  d.append(pd.read_csv(i, skiprows=errors_, names=['word','values_']))  

In [12]:
len(d)

0

In [13]:
glove_clean=[d[i][d[i].iloc[:,0].str.len()<20] for i in range(len(d))]

In [15]:
!ls

drive  sample_data


In [16]:
for j in range(len(glove_clean)):
	glove_clean[j].to_csv(f'nlp_deep_learning/glove_clean/glove_file_{j}.csv')

In [22]:
import time
import pandas as pd
import numpy as np
import glob
import torch
import itertools
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn import preprocessing
from numpy import zeros

In [23]:
data3=pd.read_csv('nlp_deep_learning/d_group2.csv')
data3=data3.iloc[:,1:]

data3=data3[(data3.occ_group=='technical')|(data3.occ_group=='complex problem solving')]

data3=data3.drop_duplicates(keep='first')

data3['occ_group']=data3['occ_group'].fillna('None')
data3=data3[data3.occ_group!='None']

In [24]:
datalist1=glob.glob('nlp_deep_learning/glove_clean/*.csv')

In [ ]:
len(datalist1)

In [26]:
combined_datalist=[pd.read_csv(i) for i in datalist1]

In [27]:
len(combined_datalist)

134

In [28]:
words_=list(itertools.chain.from_iterable([list(i.word) for i in combined_datalist]))
values_=list(itertools.chain.from_iterable([list(i.values_) for i in combined_datalist]))

In [42]:
values_=[' '.join(s) for s in [list(re.findall(r'[\d\.]{1,8}',str(i))) for i in values_]]

In [43]:
print(values_[0])

0.038194 0.24487 0.72812 0.39961 0.083172 0.043953 0.39141 0.3344 0.57545 0.087459 0.28787 0.06731 0.30906 0.26384 0.13231 0.20757 0.33395 0.33848 0.31743 0.48336 0.1464 0.37304 0.34577 0.052041 0.44946 0.46971 0.02628 0.54155 0.15518 0.14107 0.039722 0.28277 0.14393 0.23464 0.31021 0.086173 0.20397 0.52624 0.17164 0.082378 0.71787 0.41531 0.20335 0.12763 0.41367 0.55187 0.57908 0.33477 0.36559 0.54857 0.062892 0.26584 0.30205 0.99775 0.80481 3.0243 0.01254 0.36942 2.2167 0.72201 0.24978 0.92136 0.034514 0.46745 1.1079 0.19358 0.074575 0.23353 0.052062 0.22044 0.057162 0.15806 0.30798 0.41625 0.37972 0.15006 0.53212 0.2055 1.2526 0.071624 0.70565 0.49744 0.42063 0.26148 1.538 0.30223 0.073438 0.28312 0.37104 0.25217 0.016215 0.017099 0.38984 0.87424 0.72569 0.51058 0.52028 0.1459 0.8278 0.27062


In [46]:
embed_index=dict(zip(words_,values_))

In [47]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(data3.no_stopwords_stemmed)

embedded_skill=word_tokenizer.texts_to_sequences(data3.no_stopwords_stemmed)

vocab_length = len(word_tokenizer.word_index) + 1

embedding_matrix = zeros((vocab_length, 100))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embed_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

ValueError: ignored

In [38]:

        
length_long_sentence=max(len_sent)

padded_sentences = pad_sequences(embedded_skill, length_long_sentence, padding='post')

validation_split=.2
indices=np.arange(np.array(padded_sentences,dtype=object).shape[0])
np.random.shuffle(indices)   

data_rand=padded_sentences[indices]

le = preprocessing.LabelEncoder()
labe=[str(i).replace('.xls','') for i in data3.occ_group]

labels_encoded=le.fit(labe)
labels_=le.transform(labe)

labels_rand=np.array(labels_)[indices]

val_sample=int(validation_split * data3.shape[0])
X_train=data_rand[:-val_sample]
y_train=labels_rand[:-val_sample]
X_test=data_rand[-val_sample:]
y_test=labels_rand[-val_sample:]

x_train = torch.tensor(X_train, dtype=torch.long)
y_train = torch.tensor(y_train, dtype=torch.long)
x_cv = torch.tensor(X_test, dtype=torch.long)
y_cv = torch.tensor(y_test, dtype=torch.long)

train = torch.utils.data.TensorDataset(x_train, y_train)
valid = torch.utils.data.TensorDataset(x_cv, y_cv)

train_loader = torch.utils.data.DataLoader(train, batch_size=70, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid, batch_size=70, shuffle=False)
class BiLSTM(nn.Module):
    
    def __init__(self):
        super(BiLSTM, self).__init__()
        self.hidden_size = 164
        drp = 0.4
        n_classes = len(le.classes_)
        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = True
        self.lstm = nn.LSTM(embed_size, self.hidden_size, bidirectional=True, batch_first=True)
        self.linear = nn.Linear(self.hidden_size*4 , 164)
        self.relu = nn.ReLU()
        self.out = nn.Linear(164, n_classes)


    def forward(self, x):
        h_embedding = self.embedding(x)
        h_lstm, _ = self.lstm(h_embedding)
        avg_pool = torch.mean(h_lstm, 1)
        max_pool, _ = torch.max(h_lstm, 1)
        conc = torch.cat(( avg_pool, max_pool), 1)
        conc = self.relu(self.linear(conc))
        out = self.out(conc)
        return out
        
        
embed_size=100
max_features=vocab_length
n_epochs = 7
model = BiLSTM()
loss_fn = nn.CrossEntropyLoss(reduction='sum')
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=0.002, weight_decay=0)
model.cpu()


val_preds1=[]
X1=[]
Y1=[]
ind1=[]
val_loss1=[]
output=[]

for epoch in range(n_epochs):
    start_time = time.time()
    
    model.train()
    avg_loss = 0.  
    for i, (x_batch, y_batch) in enumerate(train_loader):
        
        y_pred = model(x_batch)
       
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avg_loss += loss.item() / len(train_loader)
    
    
    model.eval()        
    avg_val_loss = 0.
    val_preds = np.zeros((len(x_cv),len(le.classes_)))
    X=np.zeros((len(x_cv),length_long_sentence))
    Y=np.zeros((len(x_cv),))
    ind=np.zeros((len(x_cv),))
    val_loss=np.zeros((len(x_cv),))
    
    for i, (x_batch, y_batch) in enumerate(valid_loader):
        
        y_pred = model(x_batch).detach()
        avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
        
        val_preds[i * 70:(i+1) * 70] =F.sigmoid(y_pred).cpu().numpy()
        X[i * 70:(i+1) * 70]=x_batch.cpu().numpy()
        Y[i * 70:(i+1) * 70]=y_batch.cpu().numpy()
        ind[i * 70:(i+1) * 70]=i
        val_loss[i * 70:(i+1) * 70]=avg_val_loss
        
    val_preds1.append(val_preds)
    X1.append(X)
    Y1.append(Y)
    ind1.append(ind)
    val_loss1.append(val_loss)
    
    val_accuracy = sum(val_preds.argmax(axis=1)==y_test)/len(y_test)
    
    
    
    elapsed_time = time.time() - start_time 
    output.append('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f}  \t val_acc={:.4f}  \t time={:.2f}s'.format(
                epoch + 1, n_epochs, avg_loss, avg_val_loss, val_accuracy, elapsed_time))
                
output_df=pd.DataFrame([i.split('\t') for i in output])
output_df.columns=['epoch','loss','validation_loss','validation_accuracy','time']

Y1_s=[int(i) for i in Y1[3]]
pred=list([i.argmax() for i in val_preds1[3]])

df_encoded=pd.concat([pd.Series(Y1_s, name='actual_class'), 
             pd.Series(pred, name='predicted_class'),
             pd.Series([list(i) for i in X1[3]], name='text')], axis=1)
             
             
word_encoding=word_tokenizer.word_index
word_decoding=dict(zip(list(word_encoding.values()), list(word_encoding.keys())))
word_decoding[0]='None'

skill_decoding=dict(zip(list(labels_), list(labe)))

text2=[]
for i in df_encoded.text:
    text2.append([word_decoding[p] for p in i])
    
text2=[i for i in text2 if i!='None']
df_encoded['text2']=pd.Series(text2)
df_encoded['actual_class_']=[skill_decoding[i] for i in df_encoded['actual_class']]
df_encoded['predicted_class_']=[skill_decoding[i] for i in df_encoded['predicted_class']]
df_encoded_=df_encoded[['actual_class_','predicted_class_','text2']].reset_index()

df_encoded_.columns=['id', 'actual_class_', 'predicted_class_', 'words']

df_encoded_            
             

     



ValueError: ignored